In [1]:
%load_ext autoreload

import os
import certifi

# we need to set gdal and rasterio defaults. this will need thought.
os.environ['GDAL_DATA']  = r'C:\Program Files\ArcGIS\Pro\Resources\pedata\gdaldata' # in order for rasterio gdal affine and crs to work
os.environ.setdefault("CURL_CA_BUNDLE", certifi.where()) # in order for xr open_rasterioor rasterion.open to work

import sys
import gdal
import rasterio
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

import dask
import dask.array as da

sys.path.append(r'C:\Users\Lewis\Desktop')
import work

delete this!
remove this


In [2]:
gdf_bounds = [
    118.928375244140611,
    -22.816061209792938,
    119.165267944335921,
    -22.681182933819271
]

In [3]:
def fetch_stac_items(bbox, start_date, end_date, collections, limit, stac_end_url):
    
    # imports
    from satsearch import Search

    # search stac
    search = Search(bbox=bbox,
                datetime='{}/{}'.format(start_date, end_date),
                collections=collections,
                limit=limit,
                url=stac_end_url)

    # notify number of items
    print('Found {} items for platform: {}'.format(search.found(), collections))
    
    # get items
    items = search.items(limit=limit)
    
    # do some other
    
    return items

url = 'https://explorer.sandbox.dea.ga.gov.au/stac/'
collections = ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']

items = fetch_stac_items(bbox=gdf_bounds, 
                         start_date='1990-01-01', 
                         end_date='1999-12-31', 
                         collections='ga_ls5t_ard_3', 
                         limit=1000, 
                         stac_end_url=url)

Found 236 items for platform: ga_ls5t_ard_3


In [4]:
# set inputs
assets=['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1', 'nbart_swir_2', 'oa_fmask']
bounds_latlon=gdf_bounds
bounds=None
epsg=3577
resolution=30
snap_bounds=True

In [5]:
# first, convert stac items to plain dicts
# plain_items = items_to_plain
# just does this:
plain_items = [item._data for item in items]

In [6]:
# sort whole dict by datetime via sorted func
plain_items = sorted(
    plain_items,
    key=lambda item: item["properties"].get('datetime', ''))

In [7]:
# START PREPARE FUNC
# now we call prepare itmes function
out_epsg = epsg
out_bounds = bounds # not bounds_latlon, this gets none if latlon bounds used

if resolution is not None and not isinstance(resolution, tuple):
    resolution = (resolution, resolution)
out_resolutions_xy = resolution

# dont need to do all the work with assets, we always want same
asset_ids = assets

# creates a table with structure for 
ASSET_TABLE_DT = np.dtype([("url", object), ("bounds", "float64", 4)])
asset_table = np.full((len(plain_items), len(asset_ids)), None, dtype=ASSET_TABLE_DT) # fills numpy array of shape (3,) filled with nan. holds scene info

# if items empty, throw error
if len(plain_items) == 0:
    raise ValueError("No items")

In [8]:
from functools import lru_cache
import pyproj

@lru_cache(maxsize=32) # this ensures we only really calc once if same epsgs
def cached_transform(from_epsg, to_epsg, skip_equivalent, always_xy):
    return pyproj.Transformer.from_crs(from_epsg, 
                                       to_epsg, 
                                       skip_equivalent=True, 
                                       always_xy=True)

In [9]:
def bounds_from_affine(af, ysize, xsize, from_epsg, to_epsg):   
    
    ul_x, ul_y = af * (0, 0)
    ll_x, ll_y = af * (0, ysize)
    lr_x, lr_y = af * (xsize, ysize)
    ur_x, ur_y = af * (xsize, 0)

    xs = [ul_x, ll_x, lr_x, ur_x]
    ys = [ul_y, ll_y, lr_y, ur_y]

    if from_epsg != to_epsg:
        #transformer = pyproj.Transformer.from_crs(from_epsg, 
                                                  #to_epsg, 
                                                  #skip_equivalent=True, 
                                                  #always_xy=True)
        transformer = cached_transform(from_epsg, 
                                       to_epsg, 
                                       skip_equivalent=True, 
                                       always_xy=True)
        
        xs_proj, ys_proj = transformer.transform(xs, ys, errcheck=True)
    else:
        xs_proj = xs
        ys_proj = ys

    return min(xs_proj), min(ys_proj), max(xs_proj), max(ys_proj)

In [10]:
def bounds_overlap(*bounds):
    min_xs, min_ys, max_xs, max_ys = zip(*bounds)
    return max(min_xs) < min(max_xs) and max(min_ys) < min(max_ys)

In [11]:
def snapped_bounds(bounds, resolutions_xy):
    import math
    
    minx, miny, maxx, maxy = bounds
    xres, yres = resolutions_xy

    minx = math.floor(minx / xres) * xres
    maxx = math.ceil(maxx / xres) * xres
    miny = math.floor(miny / yres) * yres
    maxy = math.ceil(maxy / yres) * yres

    return (minx, miny, maxx, maxy)

In [12]:
# start working items
for item_i, item in enumerate(plain_items):
    item_epsg = item['properties'].get("proj:epsg")
    item_bbox = item['properties'].get("proj:bbox")
    item_shape = item['properties'].get("proj:shape")
    item_transform = item['properties'].get("proj:transform")
    
    item_bbox_proj = None
    for asset_i, a_id in enumerate(asset_ids):
        try:
            asset = item['assets'].get(a_id)
        except KeyError:
            continue
            
        asset_epsg = asset.get("proj:epsg", item_epsg)
        asset_bbox = asset.get("proj:bbox", item_bbox)
        asset_shape = asset.get("proj:shape", item_shape)
        asset_transform = asset.get("proj:transform", item_transform)
        asset_affine = None
        
        # stackstac has logic to use scene crs if projection not provided. we always want to project, so ignore
        out_epsg = int(out_epsg)
        
        # project bounds to requested epsg
        if bounds_latlon is not None and out_bounds is None:
            
            with rasterio.Env(CURL_CA_BUNDLE=certifi.where()) as env:
                from rasterio.warp import transform_bounds

                # convert selected bounding box to epsg in output scene, liek so V
                l, b, r, t = bounds_latlon[0], bounds_latlon[1], bounds_latlon[2], bounds_latlon[3]
                out_bounds = bounds = transform_bounds(src_crs=4326, 
                                                       dst_crs=out_epsg, 
                                                       left=l, bottom=b, right=r, top=t)
        
        # if asset bbox exists, use that, else use scene bbox
        # not doing that
        # use asset transform instead
        import affine
        if asset_transform is not None and asset_shape is not None and asset_epsg is not None:
            asset_affine = affine.Affine(*asset_transform[:6]) # get affine
            
            # check the lru_cache thing here in stackstac, might go faster
            asset_bbox_proj = bounds_from_affine(asset_affine,
                                                 asset_shape[0],
                                                 asset_shape[1],
                                                 asset_epsg,
                                                 out_epsg)
            
        else:
            raise ValueError('No scene transform')
        
        # create bounds. simplified this
        if bounds is None:
            if asset_bbox_proj is None:
                raise ValueError('not enough spatial info')
            
            if out_bounds is None:
                out_bounds = asset_bbox_proj
            else:
                print('code up geom_utils.union_bounds(asset_bbox_proj, out_bounds)')
                
        else:
            if asset_bbox_proj is not None and not bounds_overlap(asset_bbox_proj, bounds):
                continue
                
        # do resolution
        if resolution is None:
            print('do some work for resolution user not provided')
            
        # store information
        # creates row in array that has 1 row per scene, n columns per requested band where (url, [l, b, r, t])
        href = asset["href"].replace('s3://dea-public-data', 'https://data.dea.ga.gov.au')
        asset_table[item_i, asset_i] = (href, asset_bbox_proj)
        
print('Done!')

# now, move things over to new vars
# he casts out_bounds to a bbox object, which is just a tuple of l, r, t, b or whatever
# does same for out_resolutions_xy
# out_epsg also same, converted to int

# snap bounds?
if snap_bounds:
    out_bounds = snapped_bounds(out_bounds, out_resolutions_xy)

# converts values to a object
#spec = RasterSpec(
        #epsg=out_epsg,
        #bounds=out_bounds,
        #resolutions_xy=out_resolutions_xy,)
        
# prepare spec dictionary
trans = affine.Affine(out_resolutions_xy[0],   # xscale
                      0.0,
                      out_bounds[0],  # xoff
                      0.0,
                      -out_resolutions_xy[1],  # yscale
                      out_bounds[3],  # yoff
                     )


def get_shape(out_bounds, out_resolutions_xy):
    minx, miny, maxx, maxy = out_bounds
    xres, yres = out_resolutions_xy
    width = int((maxx - minx + (xres / 2)) / xres)
    height = int((maxy - miny + (yres / 2)) / yres)

    return (height, width)    
    

    # This is how GDAL rounds/snaps the calculation, so we do it too
    # https://github.com/OSGeo/gdal/blob/00615775bff0681a7fbce17eb187dcfc0e000c15/gdal/apps/gdalwarp_lib.cpp#L3394-L3399
    # (it's not quite the same as `round`)
    width = int((maxx - minx + (xres / 2)) / xres)
    height = int((maxy - miny + (yres / 2)) / yres)

    return (height, width)


spec = {'epsg': out_epsg,
        'bounds': out_bounds,
        'resolutions_xy': out_resolutions_xy,
        'shape': get_shape(out_bounds, out_resolutions_xy),
        'transform': trans,
        'vrt_params': {
            'crs': out_epsg,
            'transform': trans,
            'height': get_shape(out_bounds, out_resolutions_xy)[0],
            'width': get_shape(out_bounds, out_resolutions_xy)[1]
        }
       }        

# drop items/assets where must be skipped. either asset missed, or out of bounds
# same size as table
isnan_table = np.isnan(asset_table["bounds"]).all(axis=-1) # uses bounds because other object isnan doesnt work
item_isnan = isnan_table.all(axis=1)  # any items all empty?
asset_id_isnan = isnan_table.all(axis=0) # bands assets all empty?

# remove nan items... np.ix_ chooses any row and column where not nan
if item_isnan.any() or asset_id_isnan.any():
    asset_table = asset_table[np.ix_(~item_isnan, ~asset_id_isnan)]
    asset_ids = [id for id, isnan in zip(asset_ids, asset_id_isnan) if not isnan]
    items = [item for item, isnan in zip(items, item_isnan) if not isnan]
    
# returns 
#return asset_table, spec, asset_ids, plain_items

D66140B0-0277-45FF-AE18-A787BE3D4AAF:9: DeprecationWarning: skip_equivalent is deprecated.


Done!


In [13]:
from rasterio.enums import Resampling
from work import items_to_dask

arr = items_to_dask(asset_table=asset_table,
                    spec=spec,
                    chunksize=512*2,
                    dtype=np.dtype('int16'),
                    resampling=Resampling.nearest,
                    fill_value=-999,
                    rescale=True,
                    reader=None,
                    gdal_env=None,
                    errors_as_nodata=()
                   )

In [14]:
import pandas as pd
from work import to_coords, to_attrs

In [15]:
dataset = xr.DataArray(
    arr,
    *to_coords(
        plain_items,
        asset_ids,
        spec,
        xy_coords='topleft',
        properties=None,
        band_coords=True
    ),
    attrs=to_attrs(spec),
    name="stackstac-" + dask.base.tokenize(arr)
)

In [19]:
from dask.diagnostics import ProgressBar
ProgressBar().register()

#%time ds = dataset.compute()
mask = dataset.to_dataset(dim='band')['oa_fmask']
%time mask = mask.compute()

[########################################] | 100% Completed |  7.1s
[########################################] | 100% Completed |  7.1s
[########################################] | 100% Completed |  7.1s
Wall time: 7.2 s


In [23]:
ds = ds.to_dataset(dim='band')

In [20]:
mask

<xarray.DataArray 'oa_fmask' (time: 236, y: 582, x: 852)>
array([[[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 2, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],

       [[2, 2, 2, ..., 1, 1, 1],
        [2, 2, 2, ..., 1, 1, 1],
        [2, 2, 2, ..., 1, 1, 1],
        ...,
        [2, 2, 2, ..., 1, 1, 1],
        [2, 2, 2, ..., 1, 1, 1],
        [2, 2, 2, ..., 1, 1, 1]],

       [[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
...
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],

       [[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],

       [[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]]], dtype=int16)
Coordinates:
  * time     (time) datetime64[ns] 1990-02-09T01:24:45.881044 ... 1999-12-19T...
    id       (time) <U36 'ba5a9c14-693c-4367-96ad-d90c490535e7' ... '974bce17...
  * x        (x) float64 -1.329e+06 -1.329e+06 ... -1.303e+06 -1.303e+06
  * y        (y) float64 -2.509e+06 -2.509e+06 ... -2.526e+06 -2.526e+06
    epsg     int32 3577

In [24]:
ds.to_netcdf(r'D:\test.nc')